In [72]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

# Neo4j Import of GraphRAG Result Parquet files

This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization. 

You can also build your own GenAI applications using Neo4j and a number of RAG strategies with LangChain, LlamaIndex, Haystack, and many other frameworks.
See: https://neo4j.com/labs/genai-ecosystem

Here is what the end result looks like:

![](https://dev.assets.neo4j.com/wp-content/uploads/graphrag-neo4j-visualization.png)

## How does it work?

The notebook loads the parquet files from the `output` folder of your indexing process and loads them into Pandas dataframes.
It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships. 

All operations use MERGE, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement

```cypher
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [73]:
GRAPHRAG_FOLDER = "/home/fei/git/graphrag/workdir/output"

### Depedendencies

We only need Pandas and the neo4j Python driver with the rust extension for faster network transport.

In [74]:
%pip install --quiet pandas neo4j-rust-ext

24948.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [75]:
import time

import pandas as pd
from neo4j import GraphDatabase

## Neo4j Installation

You can create a free instance of Neo4j [online](https://console.neo4j.io). You get a credentials file that you can use for the connection credentials. You can also get an instance in any of the cloud marketplaces.

If you want to install Neo4j locally either use [Neo4j Desktop](https://neo4j.com/download) or 
the official Docker image: `docker run -e NEO4J_AUTH=neo4j/password -p 7687:7687 -p 7474:7474 neo4j` 

In [76]:
NEO4J_URI = "neo4j+s://e941d51d.databases.neo4j.io"  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "r2Fboc7Ul7R6MTiobAQva-rm0jYuvVtCjvbYJe_xS0A"  # your password
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)
NEO4J_DATABASE = "neo4j"

# Create a Neo4j driver
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

Connection established.


In [77]:
# NEO4J_URI = "neo4j://localhost"  # or neo4j+s://xxxx.databases.neo4j.io
# NEO4J_USERNAME = "neo4j"
# NEO4J_PASSWORD = "password"  # your password
# AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)
# NEO4J_DATABASE = "neo4j"

# # Create a Neo4j driver
# with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
#     driver.verify_connectivity()
#     print("Connection established.")

# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Batched Import

The batched import function takes a Cypher insert statement (needs to use the variable `value` for the row) and a dataframe to import.
It will send by default 1k rows at a time as query parameter to the database to be inserted.

In [78]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.

    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start : min(start + batch_size, total)]
        result = driver.execute_query(
            "UNWIND $rows AS value " + statement,
            rows=batch.to_dict("records"),
            database_=NEO4J_DATABASE,
        )
        print(result.summary.counters)
    print(f"{total} rows in {time.time() - start_s} s.")
    return total

## Indexes and Constraints

Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect.
Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [79]:
# create constraints, idempotent operation

statements = [
    "\ncreate constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique",
    "\ncreate constraint document_id if not exists for (d:__Document__) require d.id is unique",
    "\ncreate constraint entity_id if not exists for (c:__Community__) require c.community is unique",
    "\ncreate constraint entity_id if not exists for (e:__Entity__) require e.id is unique",
    "\ncreate constraint entity_title if not exists for (e:__Entity__) require e.name is unique",
    "\ncreate constraint entity_title if not exists for (e:__Covariate__) require e.title is unique",
    "\ncreate constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique",
    "\n",
]

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process

### Importing the Documents

We're loading the parquet file for the documents and create nodes with their ids and add the title property.
We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [80]:
doc_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_documents.parquet", columns=["id", "title"]
)
doc_df.head(2)

,id,title
0,647709a0d0e057670c5b0dbc015bb7ba03d6fedccb9a5f...,report.c
1,9f5a1185c3548f00b01f1686b1255c749c28fe24b4591f...,hashtable_utils.c


In [81]:
# Import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 38, 'nodes_created': 38, 'properties_set': 76}
38 rows in 0.5366926193237305 s.


38

### Loading Text Units

We load the text units, create a node per id and set the text and number of tokens.
Then we connect them to the documents that we created before.

In [82]:
text_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_text_units.parquet",
    columns=["id", "text", "n_tokens", "document_ids"],
)
text_df.head(2)

,id,text,n_tokens,document_ids
0,dcaa484d1db860309b46a809929f5c1c27e7e53e0ab92b...,#ifndef _SSDBUFTABLE_H\n#define _SSDBUFTABLE_H...,180,[0038e02189a4539ec6415c2b5d6be32a8947084c04302...
1,9f1e4d47e4e851180c13f165a57ae3143054795d165ed7...,#ifndef _STRAtEGY_H_\n#define _STRAtEGY_H_\n\n...,134,[0f329dd40b18a57b4daba0923e508b26b3a7682cb90cb...


In [83]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 67, 'relationships_created': 67, 'nodes_created': 67, 'properties_set': 201}
67 rows in 0.9977202415466309 s.


67

### Loading Nodes

For the nodes we store id, name, description, embedding (if available), human readable id.

In [84]:
entity_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_entities.parquet",
    columns=[
        "id",
        "human_readable_id",
        "title",
        "type",
        "description",
        "text_unit_ids",
    ],
)
entity_df.head(2)

,id,human_readable_id,title,type,description,text_unit_ids
0,87eaa567-cc2e-4f6b-a3a5-3d1ca18eb87c,0,SSDBUFHASHBUCKET,DATA STRUCTURE,A data structure used to store information in ...,[dcaa484d1db860309b46a809929f5c1c27e7e53e0ab92...
1,b170cde1-355e-469e-bb6e-baa9680eba21,1,SSD_BUF_HASHTABLE,VARIABLE,A variable that points to the first SSDBufHash...,[dcaa484d1db860309b46a809929f5c1c27e7e53e0ab92...


In [85]:
# entity_statement = """
# MERGE (e:__Entity__ {id:value.id})
# SET e += value {.human_readable_id, .description, title:replace(value.title,'"','')}
# WITH e, value
# CALL db.create.setNodeVectorProperty(e, "description", value.description)
# CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
# UNWIND value.text_unit_ids AS text_unit
# MATCH (c:__Chunk__ {id:text_unit})
# MERGE (c)-[:HAS_ENTITY]->(e)
# """

# batched_import(entity_statement, entity_df)

In [86]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, title:replace(value.title,'"','')}
WITH e, value
FOREACH (label IN CASE WHEN value.type IS NOT NULL AND value.type <> '' 
         THEN [value.type] ELSE [] END | 
         SET e:`${label}`)
WITH e, value
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 924, 'relationships_created': 1008, 'nodes_created': 474, 'properties_set': 1896}
474 rows in 1.448024034500122 s.


474

### Import Relationships

For the relationships we find the source and target node by name, using the base `__Entity__` type.
After creating the `RELATED` relationships, we set the description as attribute.

In [87]:
rel_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_relationships.parquet",
    columns=[
        "source",
        "target",
        "id",
        "combined_degree",
        "weight",
        "human_readable_id",
        "description",
        "text_unit_ids",
    ],
)
rel_df.head(2)

,source,target,id,combined_degree,weight,human_readable_id,description,text_unit_ids
0,SSDBUFHASHBUCKET,SSD_BUF_HASHTABLE,af0e92ee-f14c-42aa-af87-eb32d4cdfc90,11,20.0,0,ssd_buf_hashtable is a pointer to the first SS...,[dcaa484d1db860309b46a809929f5c1c27e7e53e0ab92...
1,SSDBUFHASHBUCKET,HASHTAB_GETHASHCODE,2509020c-bfe8-4a18-ab83-a01433fab524,10,7.0,1,HashTab_GetHashCode function uses SSDBufHashBu...,[dcaa484d1db860309b46a809929f5c1c27e7e53e0ab92...


In [88]:
rel_statement = """
    MATCH (source:__Entity__ {title:replace(value.source,'"','')})
    MATCH (target:__Entity__ {title:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 653, 'properties_set': 3265}
653 rows in 2.0415780544281006 s.


653

### Importing Communities

For communities we import their id, title, level.
We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [89]:
community_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_communities.parquet",
    columns=["id", "level", "title", "text_unit_ids", "relationship_ids"],
)

community_df.head(2)

,id,level,title,text_unit_ids,relationship_ids
0,73cbeacb-7f76-46d1-a025-e9e70d7c7396,0,Community 0,[0806dbf3d6f7481790423036bc2afd6b54647c4282307...,"[0309cdc2-9383-4b4e-be78-0675a75004d2, 0a9b46a..."
1,843a8571-eebf-4504-9c21-d84a5aefd271,0,Community 1,[25dc39b6f5cca06b4b96f4492bfbe343263f5f2770505...,"[38489977-ffba-4057-88fd-cf84c78e3be4, 3a0a40c..."


In [90]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 85, 'relationships_created': 766, 'nodes_created': 85, 'properties_set': 255}
85 rows in 2.3909716606140137 s.


85

### Importing Community Reports

Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about.
For the findings we create the findings in context of the communities.

In [91]:
community_report_df = pd.read_parquet(
    f"{GRAPHRAG_FOLDER}/create_final_community_reports.parquet",
    columns=[
        "id",
        "community",
        "level",
        "title",
        "summary",
        "findings",
        "rank",
        "rank_explanation",
        "full_content",
    ],
)
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,ce0cedd9eb654e48b68cf66540d34ad0,64,2,Optimizing SSD Cache Management in Software En...,This report delves into the intricate network ...,[{'explanation': 'The HIT_SAC function is pivo...,9.5,The technical depth and direct applicability o...,# Optimizing SSD Cache Management in Software ...
1,b4691d06dd5e46d996aa06d364bb3656,65,2,DSCPTR_SAC: A Keystone in Software Engineering...,The DSCPTR_SAC data structure is pivotal in th...,[{'explanation': 'The DSCPTR_SAC data structur...,9.0,The detailed structure and relationships of DS...,# DSCPTR_SAC: A Keystone in Software Engineeri...


In [92]:
# Import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 520, 'relationships_created': 435, 'nodes_created': 520, 'properties_set': 1900}
85 rows in 0.594315767288208 s.


85

### Importing Covariates

Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

In [93]:
cov_df = (pd.read_parquet(f"{GRAPHRAG_FOLDER}/create_final_covariates.parquet"),)
#                         columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids

FileNotFoundError: [Errno 2] No such file or directory: '/home/fei/git/graphrag/workdir/output/create_final_covariates.parquet'

In [ ]:
# Import covariates
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)

{'_contains_updates': True, 'labels_added': 89, 'relationships_created': 89, 'nodes_created': 89, 'properties_set': 1061}
89 rows in 0.13370895385742188 s.


89

### Visualize your data

You can now [Open] Neo4j on Aura, you need to log in with either SSO or your credentials.

Or open https://workspace-preview.neo4j.io and connect to your local instance, remember the URI is `neo4j://localhost` and `neo4j` as username and `password` as password.

In "Explore" you can explore by using visual graph patterns and then explore and expand further.

In "Query", you can open the left sidebar and explore by clicking on the nodes and relationships.
You can also use the co-pilot to generate Cypher queries for your, here are some examples.

#### Show a few `__Entity__` nodes and their relationships (Entity Graph)

```cypher
MATCH path = (:__Entity__)-[:RELATED]->(:__Entity__)
RETURN path LIMIT 200
```

#### Show the Chunks and the Document (Lexical Graph)

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(c:__Chunk__)
RETURN path LIMIT 100
```

####  Show a Community and it's Entities

```cypher
MATCH (c:__Community__) WITH c LIMIT 1
MATCH path = (c)<-[:IN_COMMUNITY]-()-[:RELATED]-(:__Entity__)
RETURN path LIMIT 100
```

#### Show everything

```cypher
MATCH (d:__Document__) WITH d LIMIT 1
MATCH path = (d)<-[:PART_OF]-(:__Chunk__)-[:HAS_ENTIY]->()-[:RELATED]-()-[:IN_COMMUNITY]->()
RETURN path LIMIT 250
```

We showed the visualization of this last query at the beginning.

If you have questions, feel free to reach out in the GraphRAG discord server: 
https://discord.gg/graphrag